# Emotify - Music Emotion Analysis

This notebook demonstrates the capabilities of the Emotify music recommendation system.

## Features
- Text-based emotion detection
- Audio feature analysis
- Music recommendation algorithms
- Data visualization

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from emotion_detection.text_emotion import TextEmotionDetector
from emotion_detection.audio_emotion import AudioEmotionDetector
from recommendation_engine.recommender import EmotionBasedRecommender
from data_processing.data_utils import DataProcessor

## 1. Text Emotion Detection

In [ ]:
# Initialize text emotion detector
text_detector = TextEmotionDetector()

# Test with sample texts
test_texts = [
    "I'm feeling really happy and excited today!",
    "This is such a sad and depressing day",
    "I need some calm and relaxing music",
    "Let's get pumped up with energetic beats!"
]

results = []
for text in test_texts:
    result = text_detector.predict(text)
    results.append({
        'text': text,
        'emotion': result['emotion'],
        'confidence': result['confidence']
    })

results_df = pd.DataFrame(results)
print(results_df)

## 2. Music Dataset Analysis

In [ ]:
# Load sample dataset
recommender = EmotionBasedRecommender('../data/datasets/sample_songs.csv')
songs_df = recommender.songs_df

print(f"Dataset shape: {songs_df.shape}")
print(f"Columns: {list(songs_df.columns)}")
print(f"\nEmotion distribution:")
print(songs_df['emotion'].value_counts())

## 3. Audio Feature Visualization

In [ ]:
# Create visualizations
plt.figure(figsize=(15, 10))

# Emotion distribution
plt.subplot(2, 3, 1)
songs_df['emotion'].value_counts().plot(kind='bar')
plt.title('Emotion Distribution')
plt.xticks(rotation=45)

# Valence vs Arousal
plt.subplot(2, 3, 2)
for emotion in songs_df['emotion'].unique():
    subset = songs_df[songs_df['emotion'] == emotion]
    plt.scatter(subset['valence'], subset['arousal'], label=emotion, alpha=0.7)
plt.xlabel('Valence')
plt.ylabel('Arousal')
plt.title('Valence-Arousal Space')
plt.legend()

# Tempo distribution
plt.subplot(2, 3, 3)
plt.hist(songs_df['tempo'], bins=20, alpha=0.7)
plt.xlabel('Tempo (BPM)')
plt.title('Tempo Distribution')

# Energy vs Danceability
plt.subplot(2, 3, 4)
plt.scatter(songs_df['energy'], songs_df['danceability'], alpha=0.7)
plt.xlabel('Energy')
plt.ylabel('Danceability')
plt.title('Energy vs Danceability')

# Correlation heatmap
plt.subplot(2, 3, 5)
audio_features = ['valence', 'arousal', 'tempo', 'energy', 'danceability']
corr_matrix = songs_df[audio_features].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlation')

plt.tight_layout()
plt.show()

## 4. Recommendation Testing

In [ ]:
# Test recommendations for different emotions
emotions = ['happy', 'sad', 'calm', 'energetic']

for emotion in emotions:
    print(f"\n=== {emotion.upper()} RECOMMENDATIONS ===")
    recommendations = recommender.recommend_by_emotion(emotion, 5)
    
    for i, song in enumerate(recommendations, 1):
        print(f"{i}. {song['title']} by {song['artist']}")
        print(f"   Valence: {song['valence']:.2f}, Arousal: {song['arousal']:.2f}")

## 5. Mood Regulation Analysis

In [ ]:
# Compare 'match' vs 'boost' strategies
target_emotion = 'sad'

print("MATCH MOOD (songs that match current sad mood):")
match_recs = recommender.recommend_by_emotion(target_emotion, 5, 'match')
for i, song in enumerate(match_recs, 1):
    print(f"{i}. {song['title']} - Emotion: {song['emotion']}")

print("\nBOOST MOOD (songs to improve sad mood):")
boost_recs = recommender.recommend_by_emotion(target_emotion, 5, 'boost')
for i, song in enumerate(boost_recs, 1):
    print(f"{i}. {song['title']} - Emotion: {song['emotion']}")